# Surfingcrypto `Portfolio`

The `surfingcrypto` portfolio.

In [82]:
%load_ext autoreload
%autoreload 2
from surfingcrypto import Config
from surfingcrypto.portfolio import Portfolio
from surfingcrypto.portfolio.portfolio_tracker import Tracker

c=Config("/Users/giorgiocaizzi/Documents/GitHub/surfingcrypto/config")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
p=Portfolio("coinbase",configuration=c)

Warning! There are 45 transactionsthat were EXCLUDED in std_df.
TransactionsHistory(Processed transactions:177, Unhandled:0, Errors:9 )


In [84]:
p.std_df.head()

,type,amount,symbol,native_amount,nat_symbol,total,subtotal,total_fee,spot_price,trade_id
datetime,,,,,,,,,,
2022-01-04T10:44:49Z,buy,1.511428,SOL,225.73,EUR,NaN,NaN,2.230,149.348800,5851e642-8786-585b-a5d7-8546c8c16129
2021-11-29T10:47:17Z,buy,1.026156,SOL,186.85,EUR,NaN,NaN,1.620,182.087335,18160e53-8019-561c-a812-efd59cc7a354
2021-08-07T14:25:58Z,sell,2737.634483,ACH,299.46,EUR,NaN,NaN,-1.440,0.109386,4dbcbf9c-cd0d-56cb-8e25-f1da9a64558f
2021-08-06T23:12:51Z,buy,2737.634483,ACH,392.16,EUR,NaN,NaN,1.475,0.143248,78ad03ab-9076-593d-87f5-524da82304dd
2021-07-31T10:18:14Z,sell,4.123547,QNT,450.91,EUR,NaN,NaN,-3.225,109.350039,6ff73f15-78ae-56e1-8a01-ce15bbf64660


## Portfolio live value tracker

In [132]:
t=Tracker(p.std_df,stocks_start="1-1-2021",configuration=c)

In [133]:
t.stocks_start

Timestamp('2021-01-01 00:00:00+0000', tz='UTC')

In [134]:
t.portfolio_df

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share
0,2022-01-04 10:44:49+00:00,buy,1.511428,SOL,225.73,149.348800
1,2021-11-29 10:47:17+00:00,buy,1.026156,SOL,186.85,182.087335
2,2021-08-07 14:25:58+00:00,sell,2737.634483,ACH,299.46,0.109386
3,2021-08-06 23:12:51+00:00,buy,2737.634483,ACH,392.16,0.143248
4,2021-07-31 10:18:14+00:00,sell,4.123547,QNT,450.91,109.350039
...,...,...,...,...,...,...
127,2021-05-01 21:45:54+00:00,buy,0.494161,ETH,1205.04,2438.557721
128,2021-05-01 19:09:11+00:00,sell,0.258429,ETH,630.96,2441.520900
129,2021-04-20 18:57:40+00:00,buy,0.258429,ETH,500.00,1906.364408
130,2018-11-09 12:05:12+00:00,sell,0.017237,BTC,93.27,5584.488743


In [135]:
print(len(t.portfolio_df.Symbol.unique()))

29


In [136]:
closedata=t.load_data()
closedata

/Users/giorgiocaizzi/Documents/GitHub/surfingcrypto/surfingcrypto/portfolio/portfolio_tracker.py:93: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  df = ts.df.loc[start:end_day, ["Close"]]


,Date,Close,symbol
0,2021-11-29 00:00:00+00:00,180.930925,SOL
1,2021-11-30 00:00:00+00:00,184.174472,SOL
2,2021-12-01 00:00:00+00:00,203.019810,SOL
3,2021-12-02 00:00:00+00:00,206.963809,SOL
4,2021-12-03 00:00:00+00:00,186.684766,SOL
...,...,...,...
2090,2018-11-05 00:00:00+00:00,5626.594402,BTC
2091,2018-11-06 00:00:00+00:00,5651.393981,BTC
2092,2018-11-07 00:00:00+00:00,5710.908071,BTC
2093,2018-11-08 00:00:00+00:00,5676.859986,BTC


In [137]:
t.error_log

[{'symbol': 'CGLD', 'error': FileNotFoundError('CGLD.csv not found.')},
 {'symbol': 'USDC', 'error': FileNotFoundError('USDC.csv not found.')}]

In [138]:
print(len(closedata.symbol.unique()))

27


In [139]:
benchmark=t.set_benchmark("ETH")
benchmark

,Date,Close
0,2021-01-01 00:00:00+00:00,600.859522
1,2021-01-02 00:00:00+00:00,638.265609
2,2021-01-03 00:00:00+00:00,796.329350
3,2021-01-04 00:00:00+00:00,848.990363
4,2021-01-05 00:00:00+00:00,894.422659
...,...,...
424,2022-03-01 00:00:00+00:00,2670.120913
425,2022-03-02 00:00:00+00:00,2654.782138
426,2022-03-03 00:00:00+00:00,2561.444538
427,2022-03-04 00:00:00+00:00,2394.415113


In [140]:
active_portfolio=t.portfolio_start_balance()
active_portfolio

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share
0,2022-01-04 10:44:49+00:00,buy,1.511428,SOL,225.73,149.348800
1,2021-11-29 10:47:17+00:00,buy,1.026156,SOL,186.85,182.087335
2,2021-08-07 14:25:58+00:00,sell,2737.634483,ACH,299.46,0.109386
3,2021-08-06 23:12:51+00:00,buy,2737.634483,ACH,392.16,0.143248
4,2021-07-31 10:18:14+00:00,sell,4.123547,QNT,450.91,109.350039
...,...,...,...,...,...,...
125,2021-05-02 04:43:56+00:00,buy,0.139772,ETH,339.30,2427.520311
126,2021-05-01 22:35:55+00:00,sell,0.143144,ETH,350.85,2451.027993
127,2021-05-01 21:45:54+00:00,buy,0.494161,ETH,1205.04,2438.557721
128,2021-05-01 19:09:11+00:00,sell,0.258429,ETH,630.96,2441.520900


In [141]:
print(len(active_portfolio.Symbol.unique()))

27


In [142]:
positions_per_day=t.time_fill()

In [143]:
positions_per_day[0]

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share,Date Snapshot
83,2018-11-09 12:06:14+00:00,buy,348.021666,BAT,93.27,0.259409,2021-01-01 00:00:00+00:00
131,2018-10-22 06:29:22+00:00,buy,0.017237,BTC,100.00,5627.999719,2021-01-01 00:00:00+00:00


In [144]:
i=0
for day in positions_per_day:
    if len(day)==0:
        i=+1
print(i)

0


In [145]:
combined_df=t.per_day_portfolio_calcs(
    positions_per_day,
    benchmark,
    closedata,
    t.stocks_start 
)
combined_df

/opt/miniconda3/envs/cryptoenv/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Adj cost,Adj cost per share,Date Snapshot,Open date,Qty,Symbol,Type,Symbol Adj Close,Adj cost daily,Benchmark Close,...,Benchmark Start Date Cost,Benchmark Return,symbol Return,symbol Share Value,Benchmark Share Value,Stock Gain / (Loss),Benchmark Gain / (Loss),Abs Value Compare,Abs Value Return,Abs. Return Compare
